# BioImagePy: metadata manipulation tutorial

In order to avoid using databases like sql to manipulate metadata for scientific data manipulation, we defined a set of rules to annotates data with text files. These text files are the metadata files stored with the `.md.json` extension: *md* for metadata and *json* for the Json format.

Data are represented using 3 levels:
1. **Data**: a single data is represented by one or several data files and one single `.md.json` file. A *Data* can be a *RawData* if it comes from an acquisition system or a *ProcessedData* if it is a *Data* obtained by processing any other *Data*,
2. **DataSet**: a collection of *Data* described in a `.md.json` file. We define two types of *DataSets*: a *RawDataSet* contains a collection of *RawData*, and a *ProcessedDataSet* contains a collection of *ProcessedData*,
3. **Experiment**: contains one *RawDataSet*, the images to be analysed, and a list of *ProcessedDataSet*, one for each process applied to the *Data*. The *Experiment* is described by a `.md.json` file.

**BioImagePy** is a python library that allows to manipulate *Data*, *Datasets* and *Experiments* with a set of high level functions without needing to manualy edit `.md.json` files.

The more common usage of **BioImagePy** is following this *Data* manipulation steps: 
1. Create an *Experiment*
2. Import data to the *Experiment*
3. Tag Data 
4. Process data. 
with goings and comming between steps 2 to 4

This first tutorial cover the step 1, 2 and 3.

## Create an experiment

To create an *Experiment* we need first to import the `experiment` module from `bioimagepy.metadata.experiment`.
Then we call the function `create` to create the *Experiment*. The three parameters are:
1. **name**: the name of the experiment
2. **author**: the person who create the experiment
3. **path**: the directory where the experiment will be create

In [ ]:
# this is an optional command to install BioImagePy from conda if you do not have yet installed BioIMagePy
import sys
!conda install --yes --prefix {sys.prefix} /anaconda3/conda-bld/osx-64/bioimagepy-0.1.0-py36_0.tar.bz2


In [ ]:
import bioimagepy.metadata.experiment as experiment

## create an experiment
my_experiment = experiment.create(name="myexperiment", 
                                  author="Sylvain Prigent", 
                                  path="./") 

print("experiment created")

The *Experiment* has been created in the same directory as this notebook in the folder *./myexperiment*. The function "list_files" below shows the actual content of the expeiment folder: 

In [ ]:
import os

def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in sorted(files):
            print('{}{}'.format(subindent, f))
            
list_files('./myexperiment')            

In the root directory of the *Experiment*, there is the `experiment.md.json` metadata file. This is the file describing the experiment. Lets read it:

In [ ]:
with open('./myexperiment/experiment.md.json', 'r') as fin:
    print(fin.read())

The file contains three 'sections':
1. **information**: contains the basic information of the *Experiment* we gave when we call the `experiment.create` function, and the createddate.
2. **rawdataset**: the directory where the *RawDataSet* metadata are stored. The *data/* directory and the `rawdatates.md.json` file have been created
3. **proceeseddatasets**: the list of *ProcessedDataSet* path, that is empty for now since no processing has been performed.

Let open the `rawdataset.md.json` file:

In [ ]:
with open('./myexperiment/data/rawdataset.md.json', 'r') as fin:
    print(fin.read())

This metadata file contains the description of the *RawDataSet*. The name of the dataset (data) and 'urls' the list of all the *RawData*. For now the dataset is empty. In the next session, we import data in the project.

## Import data

Data import into an *Experiment* can be done with two functions: `experiment.import_data()` to import a single data or `experiment.import_dir()` to import multiple data from a directory. In this tutorial, we will import the data from a directory containing a syntetic data samples. This syntetic data samples is located in the directory *'./synthetic_data/data/'* in the same directory of this notebook: 

In [ ]:
list_files('./synthetic_data/data/') 

This data set contains 40 *.tif* images separated in two populations of 20 images. These images are 128\*128 2D images containing spots in the noise. Population1 images contains an average of 3.4 spots and Population2 images contains an average of 4.7 spots. Let visualize two samples:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('./synthetic_data/data/population1_001.tif')
imgplot = plt.imshow(img)
plt.show()
img2=mpimg.imread('./synthetic_data/data/population2_001.tif')
imgplot = plt.imshow(img2)
plt.show()

To import the data in the experiment, we just need to call the `experiment.import_dir()` function from the experiment module:

In [ ]:
# import the data
experiment.import_dir(experiment=my_experiment, 
                      dir_path='./synthetic_data/data/', 
                      filter="\.tif$", 
                      author='Sylvain Prigent', 
                      datatype='image', 
                      createddate='2019-03-17', 
                      copy_data=True)

# show the experiment directory
list_files('./myexperiment') 

The parameters of the `experiment.import_dir()` function are this following: `experiment` is the experiment where we want to import the data, `dir_path` is the directory where the origin data are. `filter` is a regular expression not to import all the files from the `dir_path` but only specific ones. Here we select only the files that ends with *.tif*. The other parameters are the common metadata of the imported images. 

Finally the `copy_data` option specify if we want to copy or not the data. Here since the data are small, we decided to copy them. If we do not copy the data, only the `.md.json` files will be created. Let open the `rawdataset.md.json` metadata file:

In [ ]:
with open('./myexperiment/data/rawdataset.md.json', 'r') as fin:
    print(fin.read())

Now it contains the relative links to all the *RawData* metadata files. Let see the content of a *RawData* metadata file:

In [ ]:
with open('./myexperiment/data/population1_001.md.json', 'r') as fin:
    print(fin.read())

The *common* section contains the metadata we specify during the import and the relative url of the data file `population1_001.tif`. We can notice the *origin* section which specify that this data is of type raw.

In this tutorial, we manually read the `.md.json` files for a didactic purpose. In normal use of **BioImagePy**, we do not do it, since these file are the backend of **BioImagePy**. Now that you know how it works in the backend, we can use the high level display function of **BioImagePy**. 

To display the content (metadata, and *RawData* ) of an *Experiment*, we just need to call the `diplay()` method on an experiment:

In [ ]:
import bioimagepy.metadata.experiment as experiment

# Display the experiment
my_experiment.display()

## Tag the RawData

One of the main advantage of the **BioImageIT** storage system with `.md.json` files is to be able to add tags to each `RawData`. With this tags, we will be able to query the data. 

For example, when we will analyse the data, we will need to perform a statistical test to determine if the *Population 1* and the *Population 2* have statistically different number of spots. Without tagging, we need to write a script that list and parse the *RawData* files and sort the data to analyse results depending on the image names (starting with *population1* or *population2*). With the tagging system, we only need to query **BioImagePy** to get the data containing the label *Population1* and *Population2* separatly.

But first we need to tag the *RawData* !

Each *RawData* file store the tags of it corresponding data, and the `experiment.md.json` file store the list of tags that are used in the *Experiment*.
Tags edition can be done manually with the dedicated methods of the classes *BiRawData* and *BiExperiment* (see **BioImagePy** documentation). Nevertheless it is tedious. Thus, **BioImagePy** have a set of functions (starting with `tag\_`) to automatically tags the `RawData` from the data files names:


In [ ]:

# Add the tag 'Population' to the experiment and extract the 
# values 'population1' and 'population2' from the images names
experiment.tag_rawdata_from_name(my_experiment, "Population", ['population1', 'population2'] )
my_experiment.display()

With a single line of code we added the tag *'Population'* to the experiment and extracted the values from the images names. When we display the experiment we now see the tags added to the images.

Now, we may want to extract the identifier of each image. It means extracting the characters after the *'\_'* and before file extention in the file name:


In [ ]:
# extract the image number using the separator tag parser
experiment.tag_rawdata_using_seperator(experiment=my_experiment, tag="number", separator="_", value_position=1)  
my_experiment.display()

The function `tag_rawdata_using_seperator` create a tag called *'number'* in the *Experiment*, and get the values for each *RawData* by taging the second filename part (`value_position=1`) by parsing the filename with the separator *'\_'* (`separator="\_"`)

With only two line of code, we tags all the data with the population and the image number. Now we can query the data.
For example let extract the data from one population:

In [ ]:
# Extract RawData for population 1
data_list = experiment.query(my_experiment, "Population=population1")

# The query function returns a list of BiRawData objects, so we print the name of each selected data
for data in data_list:
    print(data.name())

If we need to process some specific data from the '*population 1*' *RawData* we can conbine queries with the **AND** keyword:


In [ ]:
# Extract RawData for population 1 with number <= 2
data_list = experiment.query(my_experiment, "Population=population2 AND number<=2")

# The query function returns a list of BiRawData objects, so we print the name of each selected data
for data in data_list:
    print(data.name())

## Wrapping up

In this tutorial, we saw the basic functionnalities of the **BioImagePy** to store and tag experimental data into a `BiExperiment` data strucure. The usualy tedious task is made easy using few python functions:
1. `experiment.create`: to create the *Experiment* metadata structure
2. `experiment.import_dir`: to import filtered data from a directory. One may use `experiment.import_data` to import a single data
3. `experiment.tag\_\*`: functions to extract tag from file names. One can also manually edit tags with specific functions (see **BioImagePy** documentation)
4. `experiment.query`: function to select some data using queries on the tags (ex: `Population=population1`)